In [1]:
'''
作業
請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras 官方文件中 fit_generator 的說明
'''

'\n作業\n請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras 官方文件中 fit_generator 的說明\n'

In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

Using TensorFlow backend.
/Users/yiheng/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yiheng/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yiheng/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yiheng/.local/lib/python3.7/site-packages/tensorflow/py

In [3]:
batch_size = 3 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 3 # 訓練的 epochs 數量

In [4]:
image_array = cifar10.load_data()

In [5]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [6]:
count = 1
def train_generator(image_array, batch_img_size=50000):
    (x_train, y_train), (x_test, y_test) = image_array
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    global count
    while True:
        for indexs in range(0, len(image_array), batch_img_size):
            x_train = x_train[indexs: indexs+batch_img_size]
            x_test = x_test[indexs: indexs+batch_img_size]
            print("count:"+str(count))
            count = count+1
            yield x_train, y_train

In [7]:
def test_generator(image_array, batch_img_size=50000):
    (x_train, y_train), (x_test, y_test) = image_array
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    global count
    while True:
        for indexs in range(0, len(image_array), batch_img_size):
            x_train = x_train[indexs: indexs+batch_img_size]
            x_test = x_test[indexs: indexs+batch_img_size]
            print("count:"+str(count))
            count = count+1
            yield x_test, y_test

In [8]:
(x_tr, y_tr), (x_ts, y_ts) = cifar10.load_data()
print('x_train shape:', x_tr.shape)
print(x_tr.shape[0], 'train samples')
print(x_ts.shape[0], 'test samples')
x_tr = x_tr.astype('float32')
x_ts = x_ts.astype('float32')
x_tr /= 255
x_ts /= 255

# Convert class vectors to binary class matrices.
y_tr = keras.utils.to_categorical(y_tr, num_classes)
y_ts = keras.utils.to_categorical(y_ts, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_tr.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit_generator(train_generator(image_array),#
                                steps_per_epoch=batch_size,
                                epochs=epochs,
                                workers=1,validation_data=(x_ts, y_ts))#

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0117 23:40:46.636328 4638739904 deprecation_wrapper.py:119] From /Users/yiheng/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)       

W0117 23:40:47.107215 4638739904 deprecation_wrapper.py:119] From /Users/yiheng/opt/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/3
count:1
count:2
count:3
count:4
count:5
count:6
count:7
count:8
count:9
count:10
count:11
count:12
3/3 [==============================] - 402s 134s/step - loss: 48.3056 - accuracy: 0.1009 - val_loss: 2.3027 - val_accuracy: 0.0977
Epoch 2/3
count:15
3/3 [==============================] - 367s 122s/step - loss: 3.9818 - accuracy: 0.1020 - val_loss: 2.3026 - val_accuracy: 0.0992
Epoch 3/3
count:18
3/3 [==============================] - 378s 126s/step - loss: 2.9762 - accuracy: 0.1070 - val_loss: 2.3026 - val_accuracy: 0.1000
Test loss: 2.302603331375122
Test accuracy: 0.10000000149011612
